# Initiliation

In [ ]:
!pip install transformers datasets --quiet

     |████████████████████████████████| 3.3 MB 9.8 MB/s 
     |████████████████████████████████| 298 kB 72.4 MB/s 
     |████████████████████████████████| 895 kB 41.3 MB/s 
     |████████████████████████████████| 61 kB 611 kB/s 
     |████████████████████████████████| 596 kB 64.9 MB/s 
     |████████████████████████████████| 3.3 MB 54.1 MB/s 
     |████████████████████████████████| 132 kB 70.8 MB/s 
     |████████████████████████████████| 243 kB 58.4 MB/s 
     |████████████████████████████████| 1.1 MB 54.7 MB/s 
     |████████████████████████████████| 271 kB 65.2 MB/s 
     |████████████████████████████████| 192 kB 71.7 MB/s 
     |████████████████████████████████| 160 kB 71.6 MB/s 


In [ ]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from datasets import load_metric

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

# Data Preparation

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd 'drive/MyDrive/Masterarbeit/Colab Notebooks/OVERVIEW MBTI/Datasets/URL_Balanced_MASK'

/content/drive/.shortcut-targets-by-id/1aHXlqhpj1STohhfU4gn53D4whaLH__Jz/Masterarbeit/Colab Notebooks/OVERVIEW MBTI/Datasets/URL_Balanced_MASK


In [ ]:
dfIE = pd.read_csv('MBTI_IE_URL_Balanced_MASK.csv', sep=",", error_bad_lines=False)
dfIE

,text,label
0,i like that you are kind as [MASK] i find that...,1
1,oh my you are right who really talks like tha...,0
2,yep yep yep especially the last one yep agree ...,0
3,things that are generalizable to the entire po...,0
4,work student hobbies studying gaming reading d...,0
...,...,...
3186,i dont recall much i just remember your willin...,1
3187,no because its complete fucking bullshit if an...,1
3188,when some days you talk with every people you ...,1
3189,it gets 100 f here in the summer only flunkies...,1


# Model Training

In [ ]:
modeltype = "princeton-nlp/sup-simcse-roberta-base"

In [ ]:
train, test = train_test_split(dfIE, test_size=0.2, random_state=0, stratify=dfIE.label)

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = AutoTokenizer.from_pretrained(modeltype)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

full_train_dataset = tokenized_train
full_eval_dataset = tokenized_test

model = AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

training_args = TrainingArguments(
    "SIMCSE_RoBERTa_IE_MASK", 
    evaluation_strategy="epoch",
    save_strategy = 'no',
    save_steps = 100000,
    save_total_limit = 1,
    metric_for_best_model="eval_f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds, labels=[0,1]))
    print(confusion_matrix(labels,preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Downloading:   0%|          | 0.00/255 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/738 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model 

# Hyperparameter Optimization

In [ ]:
! pip install optuna --quiet

     |████████████████████████████████| 308 kB 8.6 MB/s 
     |████████████████████████████████| 80 kB 7.7 MB/s 
     |████████████████████████████████| 209 kB 66.6 MB/s 
     |████████████████████████████████| 75 kB 5.1 MB/s 
     |████████████████████████████████| 112 kB 70.1 MB/s 
     |████████████████████████████████| 49 kB 6.8 MB/s 
     |████████████████████████████████| 149 kB 74.3 MB/s 


In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

In [ ]:
trainer = Trainer(
      model_init=model_init,
      args=training_args, 
      train_dataset=full_train_dataset, 
      eval_dataset=full_eval_dataset,
      compute_metrics=compute_metrics 
  )

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1480925a23f7db13cea1c830922dbd4173c2a1ccab8c57cbb36a1ea693164879.01dc297b74ef2153586ff6f1113a3309f339a11f1cef9d887ae2314924e8d17e
Model config RobertaConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-roberta-base",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.13.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "v

In [ ]:
import sklearn.metrics as metrics
import optuna
import sys
import logging

def objective (metrics):
  return metrics['eval_f1']

def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-8, 5e-1, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 5e-12, 5e-1, log=True),
        "num_train_epochs": trial.suggest_float("num_train_epochs",1,8,log=True),
        #"adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6, log=True),
        #"seed" : trial.suggest_float("seed",10,60,log=True)
        }

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "SIMCSE_RoBERTa_MASK_IE"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,compute_objective=objective, n_trials=50, direction="maximize",study_name=study_name, storage=storage_name )

study = optuna.create_study()

[I 2021-12-09 19:50:50,612] A new study created in RDB with name: SIMCSE_RoBERTa_MASK_IE


A new study created in RDB with name: SIMCSE_RoBERTa_MASK_IE


Trial:
loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1480925a23f7db13cea1c830922dbd4173c2a1ccab8c57cbb36a1ea693164879.01dc297b74ef2153586ff6f1113a3309f339a11f1cef9d887ae2314924e8d17e
Model config RobertaConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-roberta-base",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.13.0",
  "type_vocab_size": 1,
  "use_cache": tru

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.688965,0.574335,0.570157,0.577554,0.574491
1,0.692400,0.687968,0.585290,0.581087,0.589078,0.585448


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text.
***** Running Evaluation *****
  Num examples = 639
  Batch size = 8


...

              precision    recall  f1-score   support

           0       0.61      0.59      0.60       320
           1       0.60      0.63      0.62       319

    accuracy                           0.61       639
   macro avg       0.61      0.61      0.61       639
weighted avg       0.61      0.61      0.61       639

[[188 132]
 [118 201]]


[I 2021-12-10 00:03:43,583] Trial 49 pruned. 


Trial 49 pruned. 


[I 2021-12-10 00:03:43,622] A new study created in memory with name: no-name-11e2e37c-ff52-4eac-9804-b73e45098ff1


A new study created in memory with name: no-name-11e2e37c-ff52-4eac-9804-b73e45098ff1


In [ ]:
storage_name

'sqlite:///SIMCSE_RoBERTa_MASK_IE.db'

In [ ]:
study_name

'SIMCSE_RoBERTa_MASK_IE'

In [ ]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-12-10 00:03:43,720] Using an existing study with name 'SIMCSE_RoBERTa_MASK_IE' instead of creating a new one.


Using an existing study with name 'SIMCSE_RoBERTa_MASK_IE' instead of creating a new one.


In [ ]:
df

,number,value,params_learning_rate,params_num_train_epochs,params_per_device_train_batch_size,params_weight_decay,state
0,0,0.581087,4.215192e-07,1.971006,8,6.152049e-11,COMPLETE
1,1,0.332985,2.086499e-01,2.320751,16,1.526087e-08,COMPLETE
2,2,0.573493,1.135507e-06,2.081758,16,8.377336e-10,COMPLETE
3,3,0.332985,5.439340e-04,1.236895,2,2.326873e-10,COMPLETE
4,4,0.656304,3.469887e-06,4.283602,4,1.000452e-04,COMPLETE
5,5,0.333681,2.498881e-04,1.737044,2,3.842662e-04,PRUNED
6,6,0.332985,1.245444e-01,1.760301,2,1.433569e-09,PRUNED
7,7,0.332985,9.960710e-03,2.335072,2,1.637649e-10,PRUNED
8,8,0.632701,1.819641e-05,6.864353,4,6.466070e-03,PRUNED
9,9,0.333681,2.834096e-04,1.956022,2,4.112338e-09,PRUNED


In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
best_run

BestRun(run_id='22', objective=0.6911127087847238, hyperparameters={'learning_rate': 8.325171558473647e-06, 'num_train_epochs': 2.8578485716657727, 'per_device_train_batch_size': 8, 'weight_decay': 0.022140347515650492})

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_edf(study)